In [1]:
!pip install confluent-kafka==1.4.2

     |████████████████████████████████| 8.1 MB 31.8 MB/s eta 0:00:01


In [2]:
!pip install confluent-kafka[avro]==1.4.2

     |████████████████████████████████| 2.4 MB 22.2 MB/s eta 0:00:01
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43513 sha256=41e6dbd6d1d7f7bef8755355792691cca372629fc2994f746bd8710c0d6d58b0
  Stored in directory: /home/jovyan/.cache/pip/wheels/a5/f2/87/b7c4b9d5915716d94e8bf2e2f3bfbbd73bb5fe2a98677a59cb
Successfully built avro-python3


In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    if not (str(t)).startswith("_"):
        print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 9 topics:
  "ratings" with 1 partition(s)
  "confluent_rmoff_01ksql_processing_log" with 1 partition(s)


In [8]:
fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

In [9]:
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(topic))
    except Exception as e:
        print("Failed to create topic {}: {}".format(topic, e))

Failed to create topic test1p: KafkaError{code=_TIMED_OUT,val=-185,str="Failed while waiting for controller: Local: Timed out"}


# Producer and Consumer API

In this part of the class, we start producing and consuming from a topic

## Write to Topic "test1p"

In [ ]:
from confluent_kafka import Producer
import sys
conf = {'bootstrap.servers': brokers}

In [ ]:
p = Producer(conf)

#### Producer API requires a call back function to control the message delivery (Python)

In [ ]:
def delivery_callback(err, msg):
        if err:
            sys.stderr.write('%% Message failed delivery: %s\n' % err)
        else:
            sys.stderr.write('%% Message delivered to %s [%d] @ %d\n' %
                             (msg.topic(), msg.partition(), msg.offset()))

### Let's send some messages

In [ ]:
for n in range(1,10):
    try:
        # Produce line (without newline)
        print(n)
        p.produce(topic_names[0], str(n), callback=delivery_callback)
        p.flush(0)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

**Ensure nothing is in the buffered by the producer.**

In [ ]:
p.poll(0)

## Reading from Topic "test1p"

In [ ]:
from confluent_kafka import Consumer, KafkaException

In [ ]:
conf = {
    'bootstrap.servers': brokers, 
    'group.id': str(uuid4()), 
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

In [ ]:
c = Consumer(conf)

In [ ]:
c.subscribe([topic_names[0]])

In [ ]:
# Read messages from Kafka, print to stdout
try:
    while True: #Consumer run forever
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            sys.stderr.write('%% %s [%d] at offset %d with key %s:\n' %
             (msg.topic(), msg.partition(), msg.offset(), str(msg.key())))
            print(msg.value())
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
finally:
    # Close down consumer to commit final offsets.
    c.close()

## Write to Topic "test2p"

#### What is important about "test2p"?

In [ ]:
conf = {'bootstrap.servers': brokers}
p = Producer(conf)

In [ ]:
for n in range(1,10):
    try:
        # Produce line (without newline)
        print(n)
        p.produce(topic_names[1], str(n), callback=delivery_callback)
        p.poll(0)
        p.flush()
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

## Reading from Topic "test2p"

In [ ]:
conf = {
    'bootstrap.servers': brokers, 
    'group.id': str(uuid4()), 
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

In [ ]:
c = Consumer(conf)
c.subscribe([topic_names[1]])

In [ ]:
# Read messages from Kafka, print to stdout
try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            print("{} [{}] at offset {} with key  {}:  {}".format(msg.topic(), msg.partition(), msg.offset(), str(msg.key()), str(msg.value())))
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
finally:
    # Close down consumer to commit final offsets.
    c.close()

### What do you notice?

![](https://github.com/DataSystemsGroupUT/dataeng/raw/dataeng/attachments/order.png)